


### **Environment Setup:**

In [1]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

In [2]:
## Necessary for Colab, not necessary for course environment
%pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2

In [3]:
## If you encounter a typing-extensions issue, restart your runtime and try again
from langchain_nvidia_ai_endpoints import ChatNVIDIA
for models in ChatNVIDIA.get_available_models():
  print(models.id)

google/deplot
writer/palmyra-med-70b
snowflake/arctic
upstage/solar-10.7b-instruct
ibm/granite-34b-code-instruct
ibm/granite-8b-code-instruct
writer/palmyra-fin-70b-32k
google/gemma-2b
mistralai/mistral-7b-instruct-v0.3
nvidia/llama3-chatqa-1.5-70b
01-ai/yi-large
microsoft/phi-3-mini-4k-instruct
deepseek-ai/deepseek-coder-6.7b-instruct
liuhaotian/llava-v1.6-mistral-7b
google/recurrentgemma-2b
google/codegemma-1.1-7b
google/codegemma-7b
meta/codellama-70b
microsoft/phi-3-medium-128k-instruct
mistralai/codestral-22b-instruct-v0.1
nvidia/llama3-chatqa-1.5-8b
meta/llama-3.1-70b-instruct
microsoft/phi-3-small-128k-instruct
nvidia/nemotron-4-340b-instruct
seallms/seallm-7b-v2.5
microsoft/phi-3-medium-4k-instruct
microsoft/phi-3-small-8k-instruct
mistralai/mistral-large
meta/llama3-70b-instruct
meta/llama-3.1-405b-instruct
microsoft/kosmos-2
microsoft/phi-3-mini-128k-instruct
mistralai/mamba-codestral-7b-v0.1
writer/palmyra-med-70b-32k
meta/llama3-8b-instruct
aisingapore/sea-lion-7b-instruct


In [4]:

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

for model in NVIDIAEmbeddings.get_available_models():
  print(model.id)

nvidia/nv-embedqa-e5-v5
nvidia/nv-embed-v1
NV-Embed-QA
snowflake/arctic-embed-l
baai/bge-m3
nvidia/nv-embedqa-mistral-7b-v2


In [6]:
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

instruct_llm_llama = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and an user named Eric. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
conv_gen_prompt = ChatPromptTemplate.from_template(
    "Generate a conversation between a Basura chat agent and an user named {user_name}"
    "Basura is a waste management company in New York which caters its clients with theor waste production data"
    "Basura just collects the data and do not handle waste produced in any way"
    "Basura's employees manually collects data from clients properties daily"
    "Make sure the topic revolves around: {topic}"
    "Generate atleast 10 rows of conversation"
    "Example conversation: [User] Hi, my name is Eric. I'd like to know about Basura\n [Agent] Hi Eric, Basura ia an intelligent waste management company in New York...."
    " Do not add Row numbers or additional information inside the tags [User] and [Agent]"
)

conv_gen_chain = (conv_gen_prompt | instruct_llm_llama | StrOutputParser())

conversation = conv_gen_chain.invoke({"user_name": "Eric", "topic": "Waste management"})
pprint(conversation)

[User] Hi, my name is Eric. I'd like to know about Basura.

[Agent] Hi Eric, Basura is an intelligent waste management company in New York that provides its clients with 
valuable insights on their waste production data. We don't handle waste in any way, our main focus is on collecting
and analyzing data to help our clients make informed decisions.

[User] That sounds interesting. How do you collect this data?

[Agent] Our team of trained professionals visits our clients' properties on a daily basis to collect the data. We 
use a combination of manual collection methods and IoT sensors to gather information on waste generation, 
composition, and disposal patterns.

[User] I see. What kind of data can your clients expect to receive from you?

[Agent] Our clients receive regular reports on their waste production, including the types and quantities of waste 
generated, recycling rates, and suggestions for improvement. This data can help them identify areas of inefficiency
and opportunities for cost savings.

[User] That makes sense. How do you think data-driven waste management can benefit businesses and residents in New 
York?

[Agent] By having access to accurate and timely data, our clients can optimize their waste management strategies, 
reduce their environmental footprint, and save money on waste disposal costs. Moreover, it can also help the city 
as a whole to develop more effective waste management policies.

[User] I'm a business owner myself, and I'm interested in learning more about how Basura can help my company. What 
kind of support do you offer to your clients?

[Agent] We offer personalized support to all our clients, including regular reporting, data analysis, and 
recommendations for improvement. Our team is always available to answer any questions or concerns you may have, and
we also provide educational resources and workshops to help you get the most out of our services.

[User] That sounds great. How can I get started with Basura?

[Agent] Simply fill out the contact form on our website, and one of our representatives will be in touch with you 
to discuss your needs and provide a customized quote. We also offer a free trial period, so you can see the value 
of our services for yourself before committing.

[User] Awesome, I'll definitely look into it. Thanks for the information!

[Agent] You're welcome, Eric. It was my pleasure to help. If you have any more questions or need further 
assistance, don't hesitate to reach out.

In [16]:
conversation = conversation.split("\n\n")
conversation

["[User] Hi, my name is Eric. I'd like to know about Basura.",
 "[Agent] Hi Eric, Basura is an intelligent waste management company in New York that provides its clients with valuable insights on their waste production data. We don't handle waste in any way, our main focus is on collecting and analyzing data to help our clients make informed decisions.",
 '[User] That sounds interesting. How do you collect this data?',
 "[Agent] Our team of trained professionals visits our clients' properties on a daily basis to collect the data. We use a combination of manual collection methods and IoT sensors to gather information on waste generation, composition, and disposal patterns.",
 '[User] I see. What kind of data can your clients expect to receive from you?',
 '[Agent] Our clients receive regular reports on their waste production, including the types and quantities of waste generated, recycling rates, and suggestions for improvement. This data can help them identify areas of inefficiency and

## Step 2: Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


In [11]:
%pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [17]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 240 ms, sys: 9.1 ms, total: 249 ms
Wall time: 3.31 s


In [18]:
convstore.index.ntotal

14

The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [19]:
pprint(retriever.invoke("What is your name?"))

[
    Document(page_content="[User] Hi, my name is Eric. I'd like to know about Basura."),
    Document(
        page_content="[User] I'm a business owner myself, and I'm interested in learning more about how Basura can 
help my company. What kind of support do you offer to your clients?"
    ),
    Document(page_content='[User] I see. What kind of data can your clients expect to receive from you?'),
    Document(
        page_content="[Agent] You're welcome, Eric. It was my pleasure to help. If you have any more questions or 
need further assistance, don't hesitate to reach out."
    )
]

In [20]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Agent] We offer personalized support to all our clients, including regular reporting, data 
analysis, and recommendations for improvement. Our team is always available to answer any questions or concerns you
may have, and we also provide educational resources and workshops to help you get the most out of our services.'
    ),
    Document(
        page_content='[Agent] Our clients receive regular reports on their waste production, including the types 
and quantities of waste generated, recycling rates, and suggestions for improvement. This data can help them 
identify areas of inefficiency and opportunities for cost savings.'
    ),
    Document(
        page_content='[Agent] By having access to accurate and timely data, our clients can optimize their waste 
management strategies, reduce their environmental footprint, and save money on waste disposal costs. Moreover, it 
can also help the city as a whole to develop more effective waste management policies.'
    ),
    Document(
        page_content='[User] That makes sense. How do you think data-driven waste management can benefit businesses
and residents in New York?'
    )
]

## Step 3: Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [21]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [23]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

chain_llama = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm_llama
    | StrOutputParser()
)
pprint(chain.invoke("Where does Basura operate"))
print("========================================")
pprint(chain_llama.invoke("How basura collects data"))

Hello there! Basura operates as an intelligent waste management company and they are based in New York. They offer 
valuable insights on waste production data to their clients.

Basura collects data through a combination of manual collection methods and the use of IoT sensors. Their team of 
trained professionals visit clients' properties daily to gather information on waste generation, composition, and 
disposal patterns.

<br>

Noticed some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

## 4: Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context. Do not hallucinate"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"

)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm_llama | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

That doesn't seem to be related to our conversation. We were discussing how Basura can help support your business. 
Did you have any further questions about our services?

Ice cream, huh? I think I can take a pretty sweet guess... is ice cream your favorite food?

Ha ha, nice surprise! I was way off! I guessed ice cream because you mentioned earlier how excited you were about 
getting some ice cream, and I thought maybe that was a clue. But honey, huh? That's a unique favorite food! What do
you love most about honey?

Yeah, I think I do! You told me your favorite food is actually honey, right? That was a sweet surprise, by the way!
I was way off with the ice cream guess.

In [25]:
convstore.index.ntotal

22

In [26]:
# Iterate over all the chunks (documents) in the FAISS vectorstore
for i, doc in enumerate(convstore.docstore._dict.values()):
    print(f"Chunk {i+1}: {doc.page_content}")


Chunk 1: [User] Hi, my name is Eric. I'd like to know about Basura.
Chunk 2: [Agent] Hi Eric, Basura is an intelligent waste management company in New York that provides its clients with valuable insights on their waste production data. We don't handle waste in any way, our main focus is on collecting and analyzing data to help our clients make informed decisions.
Chunk 3: [User] That sounds interesting. How do you collect this data?
Chunk 4: [Agent] Our team of trained professionals visits our clients' properties on a daily basis to collect the data. We use a combination of manual collection methods and IoT sensors to gather information on waste generation, composition, and disposal patterns.
Chunk 5: [User] I see. What kind of data can your clients expect to receive from you?
Chunk 6: [Agent] Our clients receive regular reports on their waste production, including the types and quantities of waste generated, recycling rates, and suggestions for improvement. This data can help them id

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## Part 3: RAG For Document Chunk Retrieval



<br>

### Loading And Chunking Your Documents


In [30]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

print(f"Constructed Docstore with {len(convstore.docstore._dict)} chunks")

Constructed Docstore with 22 chunks


<br>

### Task 3: Implement Your RAG Chain

In [33]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

#convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a Basura chat agent here to help the user!"
    "\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a Basura document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt | instruct_llm | StrOutputParser()


retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | convstore.as_retriever()  | long_reorder | docs2str})
    #| RPrint()
)


def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about Basura!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

Hello! I'd be happy to tell you about Basura. Basura is an intelligent waste management company located in New York. Rather than handling waste, our main focus is on collecting and analyzing data related to our clients' waste production. This enables us to provide valuable insights, which in turn helps companies make well-informed decisions regarding their waste management. If you're a business owner and interested in learning more, I can guide you on how to get started with Basura!

### **Task 4:** Interact With Your Gradio Chatbot

In [34]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0aca49fe66fb858838.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0aca49fe66fb858838.gradio.live
Closing server running on port: 7860
